### 產生 Markdown 輸出

預設情況下，Crawl4AI 會自動從每個爬取的頁面產生 Markdown 檔案。不過，具體輸出結果取決於你指定了 Markdown 產生器還是內容過濾器。

- **result.markdown**:  
    - 直接HTML轉換為Markdown

- **result.markdown.fit_markdown**:  
    - 套用任何已配置的內容過濾器(PruningContentFilter)



In [1]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig,CacheMode
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

md_generator = DefaultMarkdownGenerator(
    content_filter = PruningContentFilter(threshold=0.4, threshold_type="fixed") #說明1
)

config = CrawlerRunConfig(
    cache_mode = CacheMode.BYPASS,
    markdown_generator = md_generator
)

async with AsyncWebCrawler() as crawler:
    result = await crawler.arun("https://news.ycombinator.com", config=config)
    print("raw Markdown length:",len(result.markdown.raw_markdown))
    print("Fit Markdown length:",len(result.markdown.fit_markdown))

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://news.ycombinator.com                                                                         |
✓ | ⏱: 0.92s 

[SCRAPE].. ◆ https://news.ycombinator.com                                                                         |
✓ | ⏱: 0.08s 

[COMPLETE] ● https://news.ycombinator.com                                                                         |
✓ | ⏱: 1.00s 

raw Markdown length: 17229
Fit Markdown length: 14770


### ⬆️說明1
`PruningContentFilter(threshold=0.4, threshold_type="fixed")`

簡單來說，這行程式碼在建立一個 **「內容修剪過濾器」 (Pruning Content Filter)**。

它的主要目的是**自動清理和簡化爬取下來的網頁內容**，去除不相關的「雜訊」（如廣告、導覽列、頁首、頁尾、側邊欄等），只保留最核心、最有價值的正文內容。

讓我們來拆解一下它的參數：

- `PruningContentFilter`: 這是一個類別，代表我們要使用的過濾器類型是「修剪過濾器」。
- `threshold=0.4`:
    - **閾值/門檻值**。`crawl4ai` 在內部會對網頁的各個區塊（block）進行評分，計算其「內容密度」或「重要性」。
    - 這個 `0.4` 就是一個判斷標準。任何評分**低於 0.4** 的區塊都會被視為「雜訊」並被**移除**。
    - 您可以調整這個值：
        - **提高閾值** (例如 `0.6`)：過濾會更嚴格，移除更多內容，可能只剩下最核心的文章段落。
        - **降低閾值** (例如 `0.2`)：過濾會更寬鬆，保留更多內容，但也可能包含一些次要資訊。
- `threshold_type="fixed"`:
    - **閾值類型為「固定的」**。這表示 `0.4` 是一個絕對的、固定的判斷標準，直接應用於所有內容區塊的評分上。這是最常用和最直觀的設定。

**總結：** 這行程式碼的意思是：「建立一個內容過濾器，它會移除所有重要性評分低於 0.4 的網頁區塊。」

In [8]:
import asyncio
from crawl4ai import (
    AsyncWebCrawler,
    CrawlerRunConfig,
    PruningContentFilter,
    CacheMode
)

async def main():
    # 1. 建立我們定義的內容過濾器
    #    這個過濾器會移除評分低於 0.4 的區塊
    mg_generator = DefaultMarkdownGenerator(
        content_filter = PruningContentFilter(threshold=0.4, threshold_type="fixed")
    )
    

    # 2. 建立一個爬蟲執行設定 (CrawlerRunConfig)
    #    - 繞過快取，確保每次都抓取最新內容
    #    - 將我們建立的過濾器加入 content_filters 列表中
    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        markdown_generator = mg_generator
    )

    # 建立爬蟲實體
    async with AsyncWebCrawler() as crawler:
        # 3. 執行爬蟲，並傳入我們的執行設定
        #    這裡以一個新聞文章頁面為例，它通常包含很多非正文內容
        result = await crawler.arun(
            url='https://cn.nytimes.com/world/20250625/iran-nuclear-sites/zh-hant/',
            config=run_config
        )

        # 4. 比較過濾前後的結果
        print("--- 原始 Markdown (未過濾) ---")
        print(result.markdown[:500] + "...") # 只顯示前 500 字元
        print("\n" + "="*50 + "\n")

        print("--- 修剪後的 Markdown (已過濾) ---")
        # 使用 .fit_markdown 來取得過濾後的結果
        print(result.markdown.fit_markdown)


if __name__ == "__main__":
    await main()
 


[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://cn.nytimes.com/world/20250625/iran-nuclear-sites/zh-hant/                                    |
✓ | ⏱: 0.41s 

[SCRAPE].. ◆ https://cn.nytimes.com/world/20250625/iran-nuclear-sites/zh-hant/                                    |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://cn.nytimes.com/world/20250625/iran-nuclear-sites/zh-hant/                                    |
✓ | ⏱: 0.44s 

--- 原始 Markdown (未過濾) ---
  * [國際 ](https://cn.nytimes.com/world/ "國際")
  * [中國 ](https://cn.nytimes.com/china/ "中國")
  * [商業與經濟 ](https://cn.nytimes.com/business/ "商業與經濟")
  * [鏡頭 ](https://cn.nytimes.com/lens/ "鏡頭")
  * [科技 ](https://cn.nytimes.com/technology/ "科技")
  * [科學 ](https://cn.nytimes.com/science/ "科學")
  * [健康 ](https://cn.nytimes.com/health/ "健康")
  * [教育 ](https://cn.nytimes.com/education/ "教育")
  * [文化 ](https://cn.nytimes.com/culture/ "文化")
  * [風尚 ](https://cn.nytimes.com/style/ "風尚")
  * [旅遊 ](https://...


--- 修剪後的 Markdown (已過濾) ---
### 國際
  * [中文 中](javascript:;)
  * [中英雙語 雙語](https://cn.nytimes.com/world/20250625/iran-nuclear-sites/zh-hant/dual/)
  * [英文 英](https://www.nytimes.com/2025/06/24/us/politics/iran-nuclear-sites.html)


JULIAN E. BARNES, HELENE COOPER, ERIC SCHMITT, RONEN BERGMAN, MAGGIE HABERMAN, JONATHAN SWAN2025年6月25日
週日的新聞發布會上，記者紛紛拍攝美國的伊朗核設施轟炸行動——「午夜之錘」——的圖示。 Alex Brandon/Associated Press
據幾名知情官員透露，美國一份初步的機密報告顯示，美國對伊朗三處核設施的轟炸僅使該國核計劃進度倒退數月。
這些官員表示，初步